# Forename

## Docker

### Problem with Docker

Issue solution description: https://github.com/docker/for-win/issues/1825

cd "C:\Program Files\Docker\Docker"
./DockerCli.exe -SwitchDaemon

### Run Docker

To get started, we need to create and mount a volume to access the query_modules directory. This directory contains all of the built-in query modules and it’s where we can save new custom query modules. Create an empty directory modules on your host machine and execute the following command:

```sh
docker volume create --driver local --opt type=none --opt device=~modules --opt o=bind modules
```

Now, you can start Memgraph and mount the created volume:
```sh
docker run -it --rm -v c:/modules:/mage/dist -p 7687:7687 -e MEMGRAPH="-query-execution-timeout-sec=0" memgraph
```

## Configuration

In [41]:
import mgclient
import pandas as pd

from bs4 import BeautifulSoup
from urllib.request import urlopen
from urllib.parse import quote 

forenames_source = r'C:\Work\Memgraph\Forename\Import\card_forenames.csv'
forenames_web = r'C:\Work\Memgraph\Forename\Import\web_forenames.csv'

export_path = 'C:/Work/Memgraph/Forename/Export/'

## Prepare database

In [42]:
# Make a connection to the database
connection = mgclient.connect(host='127.0.0.1', port=7687)
connection.autocommit = True
# Create a cursor for query execution
cursor = connection.cursor()

In [43]:
# Delete database
cursor.execute("""MATCH (n) DETACH DELETE n;""")
cursor.fetchone()

# Create database indexes and constraint
cursor.execute("""CREATE INDEX ON :Forename(value);""")
cursor.fetchone()
cursor.execute("""CREATE INDEX ON :Forename(degree);""")
cursor.fetchone()
cursor.execute("""CREATE INDEX ON :Forename(valid);""")
cursor.fetchone()
cursor.execute("""CREATE INDEX ON :Forename(normalizedValue);""")
cursor.fetchone()
cursor.execute("""CREATE CONSTRAINT ON (n:Forename) ASSERT n.value IS UNIQUE;""")
cursor.fetchone()

## Load forenames from source

In [44]:
df_forenames = pd.read_csv(forenames_source, sep=',')
df_forenames.head(5)

batch = df_forenames.values.tolist()

In [45]:
query = """
UNWIND $batch as row
WITH row[0] AS degree,
     row[1] AS forename

CREATE (f:Forename {value: forename}) SET f.degree = degree
"""

# Execute the query
cursor.execute(query, {'batch': batch})

## Data scraping

### Data scraping - Forename, gender, name day: https://www.kurzy.cz/kalendar/svatky/abecedni-seznam-jmen/

In [46]:
df_forenames = pd.read_csv(forenames_web, sep=';')
df_forenames.head(5)

batch = df_forenames.values.tolist()

In [47]:
query = """
UNWIND $batch as row
WITH row[0] AS forename,
     row[1] AS nameDay,
     row[2] AS gender

MATCH (f:Forename {value: forename})
SET f.valid = true, f.gender = gender, f.nameDay = nameDay, f.nameDayDay = SPLIT(nameDay, '.')[0], f.nameDayMonth = SPLIT(nameDay, '.')[1]
SET f.source = ',www.kurzy.cz'
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query, {'batch': batch})

### Data scraping - Forename, gender, name day and nick names: http://www.e-horoskopy.cz/vyznam-jmen.asp

In [48]:
def get_forename_detail(forename):
    url = r'http://www.e-horoskopy.cz/vyznam-jmen.asp?searchtext=' + quote(forename.encode('Windows-1250'))

    soup = BeautifulSoup(urlopen(url))   

    table = soup.find("table")
    rows = table.findAll('tr')

    final_list = []
    intermediate_list = []

    for tr in rows:
        for td in tr.findAll("td"):
            intermediate_list.append(td.findNext(text=True))

        final_list.append(intermediate_list)
        intermediate_list = []

    return final_list

In [49]:
forename_list = []

soup = BeautifulSoup(urlopen('http://www.e-horoskopy.cz/seznam-jmen.asp'))

for forename in soup.find_all('a'): 
    if 'Seznam jmen' in  forename.findParent().get_text():
        forename_list.append(forename.get_text().strip())

objects = {"objects":[]}

for forename in forename_list:
    detail = get_forename_detail(forename)
    object = {}
    object["Jméno"] = forename
    object["Svátek"] = detail[0][1]
    object["Druh jména"] = detail[1][1]
    object["Původ"] = detail[2][1]
    object["Význam"] = detail[3][1]
    object["Domácké podoby"] = detail[4][1]
    object["Oblíbenost"] = detail[5][1]

    objects["objects"].append(object)   

df_forenames = pd.DataFrame(objects["objects"])
df_forenames.head(10)

batch = df_forenames.values.tolist()

In [50]:
query = """
UNWIND $batch as row
WITH row[0] AS forename,

     CASE 
     WHEN row[1] CONTAINS 'leden' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".1." 
     WHEN row[1] CONTAINS 'únor' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".2." 
     WHEN row[1] CONTAINS 'březen' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".3." 
     WHEN row[1] CONTAINS 'duben' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".4." 
     WHEN row[1] CONTAINS 'květen' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".5." 
     WHEN row[1] CONTAINS 'červen' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".6." 
     WHEN row[1] CONTAINS 'červenec' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".7." 
     WHEN row[1] CONTAINS 'srpen' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".8." 
     WHEN row[1] CONTAINS 'září' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".9."  
     WHEN row[1] CONTAINS 'říjen' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".10." 
     WHEN row[1] CONTAINS 'listopad' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".11." 
     WHEN row[1] CONTAINS 'prosinec' THEN TOINTEGER(SPLIT(row[1] , '.')[0]) + ".12." 
     END AS nameDay,

     CASE row[2] WHEN 'ženské' THEN 'F' WHEN 'mužské' THEN 'M' END AS gender,
     row[3] AS origin,
     row[4] AS meaning,
     row[5] AS nickNames

MATCH (f:Forename {value: forename})
SET f.valid = true, f.gender = gender, f.nickNames = nickNames, f.nameDay = nameDay, f.nameDayDay = SPLIT(nameDay, '.')[0], f.nameDayMonth = SPLIT(nameDay, '.')[1], f.origin = origin
SET f.source = f.source + ',www.e-horoskopy.cz'
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query, {'batch': batch})

### Data scraping - Forename: https://www.kdejsme.cz/seznam/

In [51]:
def get_forenames(letter):

    url = r'https://www.kdejsme.cz/seznam/jmeno/' + quote(letter.encode('utf-8'))
    soup = BeautifulSoup(urlopen(url))
    table = soup.find("div", {"class":"longDiv"})
    rows = table.findChildren('a')

    objects = {"objects":[]}

    for tr in rows:
        if tr.text != letter:
            object = {}
            object["Jméno"] = tr.text
            objects["objects"].append(object)   

    df_forenames = pd.DataFrame(objects["objects"])
    df_forenames.head(10)

    return df_forenames

In [52]:
forename_list = []

soup = BeautifulSoup(urlopen('https://www.kdejsme.cz/seznam/'))

table = soup.find("table")
rows = table.findAll('tr')

final_list = []
intermediate_list = []

for tr in rows:
    for td in tr.findAll("td"):
        intermediate_list.append(td.findNext(text=True))

    final_list.append(intermediate_list)
    intermediate_list = []

for i in final_list:
    if len(i) == 2 and i[1] != '\n':
        df_forenames = get_forenames(i[1])

        batch = df_forenames.values.tolist()

        query = """
        UNWIND $batch as row
        WITH row[0] AS forename
        WHERE row[0] IS NOT NULL

        //MERGE (f:Forename {value: forename})
        //ON CREATE SET f.valid = true, f.degree = 0
        //ON MATCH SET f.valid = true

        MATCH (f:Forename {value: forename})
        SET f.valid = true
        SET f.source = f.source + ',www.kdejsme.cz'

        RETURN COUNT(*);
        """

        # Execute the query
        cursor.execute(query, {'batch': batch})

### Data scraping - Forename, gender, name day: http://svatky.centrum.cz/jmenny-seznam/?month=1&order=na

In [53]:
# http://svatky.centrum.cz/jmenny-seznam/?gender=2&order=na&month=1 .. Female
# http://svatky.centrum.cz/jmenny-seznam/?gender=1&order=na&month=1 .. Male

In [54]:
def get_forenames(month, gender):
    url = r'http://svatky.centrum.cz/jmenny-seznam/?gender=' + str(gender) + '&order=na&month=' + str(month)
    soup = BeautifulSoup(urlopen(url))
    table = soup.find("table", {"class":"t-table list-names"})
    rows = table.findAll('tr')

    final_list = []
    intermediate_list = []

    for tr in rows:
        for td in tr.findAll("td"):
            if td.findChild('a'):
                intermediate_list.append(td.findNext('a').getText())
            else:
                intermediate_list.append(td.findNext(text=True))

        final_list.append(intermediate_list)
        intermediate_list = []

    df_forenames = pd.DataFrame(final_list)
    # Drop first row
    df_forenames = df_forenames.iloc[1: , :]
    df_forenames.columns = ["nameDay", "forename", "popularity", "order"]

    df_forenames['gender'] = gender
    df_forenames.head(10)

    return df_forenames

In [55]:
for month in range(1, 13):
    for gender in range(1, 3):
        df_forenames = get_forenames(month, gender)

        batch = df_forenames.values.tolist()

        query = """
        UNWIND $batch as row
        WITH row[0] AS nameDay,
             row[1] AS forename,
             CASE row[4] WHEN 1 THEN 'F' WHEN 2 THEN 'M' END AS gender

        MATCH (f:Forename {value: forename})
        SET f.valid = true, f.gender = gender, f.nameDay = nameDay, f.nameDayDay = SPLIT(nameDay, '.')[0], f.nameDayMonth = SPLIT(nameDay, '.')[1]
        SET f.source = f.source + ',www.svatky.centrum.cz'
        RETURN COUNT(*);
        """

        # Execute the query
        cursor.execute(query, {'batch': batch})

## Forename value normalization

In [56]:
# Forename value normalization
query = """
MATCH (f:Forename)

CALL text_util.normalizeStr(f.value, 'cz') YIELD normalizedStr
SET f.normalizedValue = normalizedStr
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

## Forename count of numbers in forename

In [57]:
query = """
// Count of numbers
MATCH (f:Forename)

CALL text_util.getNumbersFromStr(f.value) YIELD numbers
SET f.valueNumberCount = size(numbers)
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

## Create similarity relations - SIMILAR_FORENAME_COMPARED_STRING 

In [58]:
query = """
// Create relationships SIMILAR_FORENAME_COMPARED_STRING
MATCH (c1:Forename) WHERE c1.valid = true
WITH c1 AS c1
MATCH (c2:Forename)
WITH c1, c2 AS c2
WHERE ID(c1) <> ID(c2) 
WITH c1, c2
WHERE c1.normalizedValue = c2.normalizedValue

WITH c1, c2

MERGE (c1)-[:SIMILAR_FORENAME_COMPARED_STRING {score: 1, method: 'By Forename - Compare normalized values'}]->(c2);
"""

# Execute the query
cursor.execute(query)

## Anonymization
After data scraping (we have valid forenames) and after SIMILAR_FORENAME_COMPARED_STRING

### Anonymization - Set anonymized = false

In [59]:
# Set anonymized = false for valid foremames
query = """
MATCH (f:Forename) 
WHERE f.valid = true
SET f.anonymized = false
RETURN COUNT(*)
"""

# Execute the query
cursor.execute(query)

# Set anonymized = false for foremames that are similar by string, SIMILAR_FORENAME_COMPARED_STRING relation exists
query = """
MATCH (f:Forename)-[r]-(:Forename)
WHERE f.valid IS NULL 
AND TYPE(r) CONTAINS 'SIMILAR_FORENAME_'
SET f.anonymized = false
RETURN COUNT(*)
"""

# Execute the query
cursor.execute(query)

### Anonymization - Do it

In [60]:
# Execute the query
query = """
MATCH (f:Forename)
WHERE f.valueNumberCount > 2

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 0
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL
  AND size(f.normalizedValue) > 11

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 1
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL
  AND (f.value CONTAINS ',')

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 2
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL
  AND (f.value CONTAINS ' ')

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 3
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL
  AND (f.value CONTAINS '-')

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 4
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL
  AND (f.value CONTAINS '_')

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 5
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL
  AND (f.value CONTAINS '%')

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 6
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL
  AND (f.value CONTAINS '@')

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 7
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL 
  AND f.degree < 5 AND SIZE(f.value) > 10

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 8
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL 
  AND f.degree < 4

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 9
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename)
WHERE f.valid IS NULL AND f.anonymized IS NULL 
  AND size(f.normalizedValue) < 3

CALL text_util.uuid_generate() YIELD uuid
SET f.value = uuid, f.normalizedValue = uuid, f.anonymized = true, f.anonymizationRule = 10
RETURN COUNT(*);
"""

# Execute the query
cursor.execute(query)

query = """
MATCH (f:Forename) 
RETURN f.valid AS valid, f.anonymized AS anonymized, f.anonymizationRule AS anonymizationRule, SUM(f.degree) AS sum_degree, COUNT(*) AS count
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['valid', 'anonymized', 'anonymizationRule', 'sum_degree', 'count'])
df.head(15)

,valid,anonymized,anonymizationRule,sum_degree,count
0,None,True,2.0,95,95
1,None,True,10.0,119,13
2,None,True,3.0,1686,1608
3,None,True,1.0,9985,9694
4,None,None,NaN,3254,143
5,None,True,4.0,182,172
6,None,True,8.0,315,261
7,None,True,5.0,3,3
8,None,True,9.0,4589,4044
9,None,True,7.0,3,3


## Create similarity relations - SIMILAR_FORENAME_LEVENSHTEIN

In [61]:
levenshteinSimilarityMin = 0.85
degree = 5

# Create relationships SIMILAR_FORENAME_LEVENSHTEIN
query = """
// Create relationships SIMILAR_FORENAME_LEVENSHTEIN
MATCH (c1:Forename) WHERE c1.degree > $degree AND c1.valid = true
WITH c1 AS c1
MATCH (c2:Forename) WHERE c2.anonymized IS NULL
WITH c1, c2 AS c2
WHERE ID(c1) <> ID(c2) 
WITH c1, c2

CALL text_util.levenshteinSimilarity(c1.normalizedValue, c2.normalizedValue) YIELD *
WITH *
WHERE score > $levenshteinSimilarityMin

MERGE (c1)-[:SIMILAR_FORENAME_LEVENSHTEIN {score: score, method: 'By Forename - Levenshtein'}]->(c2)
"""

# Execute the query
cursor.execute(query, {'levenshteinSimilarityMin': levenshteinSimilarityMin, 'degree': degree})

## Create similarity relations - SIMILAR_FORENAME_JARO

In [62]:
jaroSimilarityMin = 0.95
degree = 5

# Create relationships SIMILAR_FORENAME_JARO
query = """
// Create relationships SIMILAR_FORENAME_JARO
MATCH (c1:Forename) WHERE c1.degree > $degree AND c1.valid = true
WITH c1 AS c1
MATCH (c2:Forename) WHERE c2.anonymized IS NULL
WITH c1, c2 AS c2
WHERE ID(c1) <> ID(c2) 
WITH c1, c2

CALL text_util.jaroDistance(c1.normalizedValue, c2.normalizedValue) YIELD *
WITH *
WHERE score > $jaroSimilarityMin

MERGE (c1)-[:SIMILAR_FORENAME_JARO {score: score, method: 'By Forename - Jaro'}]->(c2)
"""

# Execute the query
cursor.execute(query, {'jaroSimilarityMin': jaroSimilarityMin, 'degree': degree})

## Create similarity relations - SIMILAR_FORENAME_JAROWINKLER

In [63]:
jaroWinklerSimilarityMin = 0.95
degree = 5

# Create relationships SIMILAR_FORENAME_JAROWINKLER
query = """
// Create relationships SIMILAR_FORENAME_JAROWINKLER
MATCH (c1:Forename) WHERE c1.degree > $degree AND c1.valid = true
WITH c1 AS c1
MATCH (c2:Forename) WHERE c2.anonymized IS NULL
WITH c1, c2 AS c2
WHERE ID(c1) <> ID(c2) 
WITH c1, c2

CALL text_util.jaroWinklerDistance(c1.normalizedValue, c2.normalizedValue) YIELD *
WITH *
WHERE score > $jaroWinklerSimilarityMin

MERGE (c1)-[:SIMILAR_FORENAME_JAROWINKLER {score: score, method: 'By Forename - JaroWinkler'}]->(c2)
"""

# Execute the query
cursor.execute(query, {'jaroWinklerSimilarityMin': jaroWinklerSimilarityMin, 'degree': degree})

## Similarity relations - Statistic

In [64]:
query = """
MATCH (c1:Forename {valid: true})-[r]->(c2) 
RETURN TYPE(r) AS type, COUNT(*) AS count, MIN(r.score) AS scoreMin, MAX(r.score) AS scoreMax, AVG(r.score) AS scoreAvg
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['type', 'count', 'scoreMin', 'scoreMax', 'scoreAvg'])
df.head(5)

,type,count,scoreMin,scoreMax,scoreAvg
0,SIMILAR_FORENAME_JARO,62,0.952381,0.966667,0.956661
1,SIMILAR_FORENAME_LEVENSHTEIN,80,0.857143,0.900000,0.869846
2,SIMILAR_FORENAME_JAROWINKLER,157,0.950000,0.987500,0.966838
3,SIMILAR_FORENAME_COMPARED_STRING,1405,1.000000,1.000000,1.000000


## Create clusters by WCC algorithm

In [65]:
# Create clusters by WCC algorithm
query = """
// WCC
CALL weakly_connected_components.get() YIELD node, component_id
WITH node, component_id

SET node.componentId = component_id
RETURN COUNT(node)
"""

# Execute the query
cursor.execute(query)

### Evaluation - Cluster

In [66]:
query = """
MATCH (n:Forename)
WHERE n.valid = true
RETURN n.componentId AS componentId, 
COUNT(n) AS count, 
COLLECT(n.value) AS forenames, 
SIZE(COLLECT(n.value)) AS forenameCount
ORDER BY count DESC
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['componentId', 'count', 'forenames', 'forenameCount'])
df.head(5)

,componentId,count,forenames,forenameCount
0,25,7,"[Michaël, Michale, Michaela, Michal, Michael, ...",7
1,7,6,"[Katarzyna, Kateřina, Katerina, Katarína, Kata...",6
2,88,6,"[Kristýna, Kristina, Krista, Kristian, Kristín...",6
3,90,6,"[L'ubomír, Lubomira, Lubomír, Lubomir, Lubomír...",6
4,205,5,"[Antonina, Antonín, Anton, Antonína, Antonin]",5


## Calculate node betweenness

In [67]:
# Calculate node betweenness centrality
query = """
// betweenness centrality
CALL betweenness_centrality.get(FALSE,FALSE)
YIELD node, betweenness_centrality

SET node.betweenness = betweenness_centrality
RETURN node.value AS forename, betweenness_centrality, node.componentId AS componentId
ORDER BY betweenness_centrality DESC
LIMIT 50
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['forename', 'betweenness_centrality', 'componentId'])
df.head(5)

,forename,betweenness_centrality,componentId
0,Michalea,138.0,25
1,Kateřina,111.0,7
2,Kristă˝na,106.0,88
3,Zdenä›k,93.0,23
4,Katarina,83.0,7


## Calculate node pageRank

In [68]:
# Calculate node pageRank
query = """
CALL pagerank.get()
YIELD node, rank

SET node.pageRank = rank
RETURN node.value AS forename, rank, node.componentId AS componentId
ORDER BY rank DESC
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['forename', 'rank', 'componentId'])
df.head(5)

,forename,rank,componentId
0,Ľudmila,0.000262,325
1,Michał,0.000262,466
2,Ĺudmila,0.000262,325
3,Gan-Erdene,0.000262,7646
4,Ľudovít,0.000262,1121


## Calculate relation bridges

In [69]:
# Calculate relation bridge 
query = """
CALL bridges.get() YIELD node_from, node_to
WITH node_from, node_to
MATCH (node_from)-[bridge]-(node_to)

SET bridge.bridge = true

RETURN bridge.bridge, node_from.value AS node_from, node_to.value AS node_to;
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['bridge', 'node_from', 'node_to'])
df.head(5)

,bridge,node_from,node_to
0,True,Drahomíra,DRAHOMÍRA
1,True,Drahomíra,drahomíra
2,True,Drahomíra,DrahomÍra
3,True,Drahomír,Drahomă­ra
4,True,Drahomir,Drahomă­ra


## Recommendation - Gender

### Create nodes Gender and LastTwoChar and relations HAS_GENDER and HAS_LAST_TWO_CHAR

In [70]:
# Create nodes Gender and LastTwoChar and relations HAS_GENDER and HAS_LAST_TWO_CHAR
cypher = '''
MATCH (f:Forename)
WHERE f.valid = true 
AND f.gender IS NOT NULL
WITH f AS f

CALL text_util.substring(f.value, -2) YIELD substring

WITH f, substring AS substring

MERGE (g:Feature:Gender {value: f.gender})
MERGE (s:Feature:LastTwoChar {value: substring})

MERGE (f)-[:HAS_GENDER]->(g)
MERGE (g)-[:HAS_LAST_TWO_CHAR]->(s)
MERGE (f)-[:HAS_LAST_TWO_CHAR]->(s)

RETURN COUNT(*)
'''

# Execute the query
cursor.execute(cypher)

In [71]:
# Calculate and set genderDegree to nodes with label LastTwoChar
cypher = '''
MATCH (n1:Feature:Gender)-[e:HAS_LAST_TWO_CHAR]->(n2:Feature:LastTwoChar)
WITH n2, COUNT(e) AS genderDegree
SET n2.genderDegree = genderDegree
RETURN COUNT(*);
'''

# Execute the query
cursor.execute(cypher)

In [72]:
cypher = '''
MATCH (f:Forename)-[:HAS_LAST_TWO_CHAR]->(fe:Feature:LastTwoChar)<-[r:HAS_LAST_TWO_CHAR]-(g:Gender)
WHERE f.gender = g.value
RETURN fe.value AS lastTwoChar, g.value AS gender, SUM(f.degree) AS sumDegree
ORDER BY lastTwoChar, gender, sumDegree DESC
'''

# Execute the query
cursor.execute(cypher)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['lastTwoChar', 'gender', 'sumDegree'])
df.head(10)

,lastTwoChar,gender,sumDegree
0,ad,M,30
1,ah,F,1
2,aj,M,449
3,al,M,11811
4,am,F,203
5,am,M,1717
6,an,F,3
7,an,M,41479
8,ar,F,3342
9,ar,M,446


In [73]:
cypher = '''
MATCH (f:Forename)-[:HAS_LAST_TWO_CHAR]->(fe:Feature:LastTwoChar)<-[r:HAS_LAST_TWO_CHAR]-(g:Gender)
WHERE f.gender = g.value
WITH fe, g, r, SUM(f.degree) AS sumDegree

SET r.degree = sumDegree
RETURN COUNT(*)
'''

# Execute the query
cursor.execute(cypher)

# Export

## Export for Tableau

### Export nodes for Tableau

In [74]:
query = """
MATCH (f1:Forename)
RETURN ID(f1) AS id, 
       f1.value AS value,
       f1.normalizedValue AS normalizedValue,
       f1.valid AS valid,
       f1.anonymized AS anonymized,
       f1.componentId AS componentId,
       f1.gender AS gender,
       f1.nameDay AS nameDay,
       f1.nameDayDay AS nameDayDay,
       f1.nameDayMonth AS nameDayMonth,
       f1.origin AS origin,
       f1.degree AS degree,
       f1.betweenness AS betweenness,
       f1.pageRank AS pageRank
ORDER BY f1.value
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['id', 'value', 'normalizedValue', 'valid', 'anonymized', 'componentId', 'gender', 'nameDay', 'nameDayDay', 'nameDayMonth', 'origin', 'degree', 'betweenness', 'pageRank'])
df.head(5)

df.to_csv(export_path + 'export_forename_nodes.csv', index = False, sep = ';')
df.head(5)

,id,value,normalizedValue,valid,anonymized,componentId,gender,nameDay,nameDayDay,nameDayMonth,origin,degree,betweenness,pageRank
0,57278,\tAnna,anna,None,False,4,None,None,None,None,None,1,0.0,0.000048
1,55551,\tBarbora,barbora,None,False,170,None,None,None,None,None,1,0.0,0.000046
2,55685,\tBeata,beata,None,False,591,None,None,None,None,None,1,0.0,0.000056
3,54244,\tDušan,dušan,None,False,109,None,None,None,None,None,1,0.0,0.000048
4,67587,\tHelena,helena,None,False,39,None,None,None,None,None,1,0.0,0.000050


### Export relationships for Tableau

In [75]:
query = """
MATCH (f1:Forename)-[r]->(f2:Forename)
RETURN ID(f1) AS id1, 
       ID(f2) AS id2, 
       f1.value AS value1,
       f2.value AS value2,
       f1.normalizedValue AS normalizedValue1,
       f2.normalizedValue AS normalizedValue2,
       f1.valid AS valid1,
       f2.valid AS valid2,
       f1.anonymized AS anonymized1,
       f2.anonymized AS anonymized2,
       f1.componentId AS componentId1,
       f2.componentId AS componentId2,
       f1.gender AS gender1,
       f2.gender AS gender2,
       f1.degree AS degree1,
       f2.degree AS degree2,
       ID(r) AS idr,
       TYPE(r) AS type,
       r.score AS score,
       r.bridge AS bridge
ORDER BY score
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['id1', 'id2', 'value1', 'value2', 'normalizedValue1', 'normalizedValue2', 'valid1', 'valid2', 'anonymized1', 'anonymized2', 'componentId1', 'componentId2', 'gender1', 'gender2', 'degree1', 'degree2', 'idr', 'type', 'score', 'bridge'])
df.head(5)

df.to_csv(export_path + 'export_forename_relations.csv', index = False, sep = ';')
df.head(5)

,id1,id2,value1,value2,normalizedValue1,normalizedValue2,valid1,valid2,anonymized1,anonymized2,componentId1,componentId2,gender1,gender2,degree1,degree2,idr,type,score,bridge
0,50735,55939,Natalie,Natăˇlie,natalie,natlie,True,None,False,None,228,228,None,None,196,7,4476,SIMILAR_FORENAME_LEVENSHTEIN,0.857143,None
1,49387,55926,Bohumír,Bohumă­r,bohumír,bohumr,True,None,False,None,213,213,M,None,140,4,4410,SIMILAR_FORENAME_LEVENSHTEIN,0.857143,None
2,49365,57273,Antonín,Antoním,antonín,antoním,True,None,False,None,205,205,M,None,2298,4,4433,SIMILAR_FORENAME_LEVENSHTEIN,0.857143,None
3,51431,55926,Bohumir,Bohumă­r,bohumir,bohumr,True,None,False,None,213,213,None,None,15,4,4404,SIMILAR_FORENAME_LEVENSHTEIN,0.857143,None
4,49391,55939,Natálie,Natăˇlie,natálie,natlie,True,None,False,None,228,228,F,None,648,7,4446,SIMILAR_FORENAME_LEVENSHTEIN,0.857143,None


## Export for Gephi

### Export nodes for Gephi

In [76]:
query = """
MATCH (f1:Forename)
RETURN ID(f1) AS Id, LABELS(f1) AS Label, f1.degree AS Interval
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['Id', 'Label', 'Interval'])
df.head(5)

df.to_csv(export_path + 'export_nodes.csv', index = False, sep = ';')
df.head(5)

,Id,Label,Interval
0,49144,[Forename],759
1,49145,[Forename],10807
2,49146,[Forename],8314
3,49147,[Forename],556
4,49148,[Forename],7040


### Export relationships for Gephi

In [77]:
query = """
MATCH (f1:Forename)-[r]-(f2:Forename)
RETURN ID(f1) AS Source, 
       ID(f2) AS Target, 
       TYPE(r) AS Type, 
       ID(r) AS Id,
       TYPE(r) AS Label,
       r.score AS Interval,
       r.score AS Weight
"""

# Execute the query
cursor.execute(query)

df = pd.DataFrame.from_records(cursor.fetchall(), columns =['Source', 'Target', 'Type', 'Id', 'Label', 'Interval', 'Weight'])
df.head(5)

df.to_csv(export_path + 'export_relations.csv', index = False, sep = ';')
df.head(5)

,Source,Target,Type,Id,Label,Interval,Weight
0,57300,49144,SIMILAR_FORENAME_COMPARED_STRING,4399,SIMILAR_FORENAME_COMPARED_STRING,1.000000,1.000000
1,68240,49144,SIMILAR_FORENAME_COMPARED_STRING,4400,SIMILAR_FORENAME_COMPARED_STRING,1.000000,1.000000
2,69960,49144,SIMILAR_FORENAME_COMPARED_STRING,4401,SIMILAR_FORENAME_COMPARED_STRING,1.000000,1.000000
3,55999,49144,SIMILAR_FORENAME_LEVENSHTEIN,4481,SIMILAR_FORENAME_LEVENSHTEIN,0.888889,0.888889
4,55999,49144,SIMILAR_FORENAME_JARO,4543,SIMILAR_FORENAME_JARO,0.962963,0.962963


# Other